In [ ]:
import numpy as np # operations on numpy arrays
import pandas as pd # dataframe
import seaborn as sns # plotting
import matplotlib.pyplot as plt # plotting

## pixel data collection for training finished on zooniverse

![title](notebook_imgs/1.zooniverse.png)

In [ ]:
# read training data
features  = np.load('Training_data/features.npy')/255 # normalize pixel value from 0-255 to 0-1
print(features.shape)

labels = np.load('Training_data/labels.npy') # background, leaf, stalk , and panicle were marked by 0, 1, 2, and 3 respectively.
print(labels.shape)

## visualize hyperspectral signatures of different plant parts

- preprocess data for plotting
- use matplotlib and seaborn - popular Data Visualization libraries in python

In [ ]:
# convert numpy array to pandas dataframe and normalize reflectance from 0-255 to 0-1
df = pd.DataFrame(features)
df.head()

In [ ]:
# load wavelength info. each wavelength respond to 1 band
df_wave = pd.read_csv('wavelength_band_info.txt', delim_whitespace=True)
df_wave.head()

In [ ]:
## add label data to the dataframe and rename each column using the corresponding wavelength
df.columns = df_wave['Wavelength(nm)']
df['Label'] = labels
df.head()

In [ ]:
# dataframe transformation to match the input data format of lineplot function in seaborn 
# details about lineplot https://seaborn.pydata.org/generated/seaborn.lineplot.html
df_melt = df.melt(id_vars='Label', value_name='Reflectance')
df_melt.head()

- this is what the [melt operation](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) looks like:
![melt operation](notebook_imgs/melt.png)

- Now the data is ready, Let's start plotting

In [ ]:
# define the colormap mapping 0,1,2,3 to different colors
from matplotlib.colors import ListedColormap
sorghum_cmap = ListedColormap(['#BBBBBB', '#32CD32', '#FF8C00', '#9400D3'], name='organs') # hex code

- you can choose your color scheme in [colorbrew](https://colorbrewer2.org/#type=qualitative&scheme=Set1&n=4)
![color scheme](notebook_imgs/color_scheme.png)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 2.5)) # define a figure and a single axes object
ax = sns.lineplot(ax=ax, data=df_melt, x='Wavelength(nm)', y='Reflectance',  
                  hue='Label', ci="sd", palette=sorghum_cmap, linewidth=1)
ax.set_xlim(530, 1711)
ax.set_yticks([0.0, 0.2, 0.4, 0.6, 0.8])
ax.set_ylabel('Normalized Intensity', fontsize=8)
ax.set_xlabel('Wavelength(nm)', fontsize=8)
ax.legend(frameon=False, fontsize=7, labels=['background','leaf','stalk','panicle'])

ax.spines['right'].set_visible(False) # remove right axis
ax.spines['top'].set_visible(False) # remove top axis
plt.tight_layout()
#plt.savefig('hyp_signature.png', dpi=300) # save figuer with resolution of 300 dpi
plt.show()

![images on cyverse](notebook_imgs/hyp_signature.png)

## training machine learning models

### PCA (Principal component analysis)

- unsupervised
- dimensionality reduction
- more details about [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA) on scikit-learn website

In [ ]:
from sklearn.decomposition import PCA # import PCA class from scikit-learn

In [ ]:
pca = PCA(n_components=2) # let's only consider the first two PCs
pca_results = pca.fit_transform(features) # Fit the model with input and apply the dimensionality reduction on it.
print(pca_results.shape)
pca_scores = pca.explained_variance_ratio_ # variance explained by each component
print('variance explained by the first two PCs: {:.2%}, {:.2%}'.format(*pca_scores))

In [ ]:
# visualize first two components
fig, ax = plt.subplots(figsize=(5,4))
s1, s2 = pca_results[:, 0], pca_results[:, 1]
scatter = ax.scatter(s1, s2, c=labels, cmap=sorghum_cmap, s=20, alpha=0.7)

# create the legend
hs, _ = scatter.legend_elements() # handlers of the legend
ls = ['background','leaf','stalk','panicle'] # labels of the legend
ax.legend(hs, ls)

# add labels
ax.set_xlabel('PC1 (%.2f)'%pca_scores[0])
ax.set_ylabel('PC2 (%.2f)'%pca_scores[1])

plt.tight_layout()
#plt.savefig('pca.png', dpi=150)
plt.show()

![images on cyverse](notebook_imgs/pca.png)

In [ ]:
from sklearn.model_selection import KFold # for five fold cross validation
from sklearn.metrics import confusion_matrix # calcuate confusion matrix

### (linear discriminant analysis)

- supervised classifier
- similar to PCA but with an extra interest on class separation
- learn more details about [LDA](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html) on scikit-learn website

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=123)
for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index] # data for training
    y_train, y_test = labels[train_index], labels[test_index] # data for validation
    
    clf = LDA() # you can specify 'n_components' option for dimensionality reduction like PCA
    clf.fit(X_train, y_train) # fit the model with both features and labels because LDA is supervised approach
    y_predict = clf.predict(X_test) # make predictions using fitted model
    acc = (y_predict==y_test).sum()/len(y_test) # calcuate the overall accuracy
    print('overal accuracy: %.3f'%acc)
    cfx = confusion_matrix(y_test, y_predict, labels=[0,1,2,3], normalize='true') # check the confusion matrix. x-axis is predicted label and y-axis is ground truth
    print('accuracy in each class: ',np.diagonal(cfx))

In [ ]:
def plot_cfx(cfx):
    '''
    visualize confusion matrix 
    '''
    fig, ax = plt.subplots(figsize=(5, 3.5))
    ax = sns.heatmap(cfx, cmap='YlGnBu', annot=True, ax=ax,
                     xticklabels=['background', 'leaf', 'stalk', 'panicle'], 
                     yticklabels=['background', 'leaf', 'stalk', 'panicle'])
    ax.set_xlabel('Predicted Label', fontsize=12)
    ax.set_ylabel('True Label',fontsize=12)
    plt.tight_layout()
    plt.show()

In [ ]:
# visualize confusion matrix
plot_cfx(cfx)

In [ ]:
# dimensionality reduction using LDA
clf = LDA(n_components=2)
lda_results = clf.fit_transform(features, labels)
ld1, ld2 = lda_results[:,0], lda_results[:,1]
lda_scores = clf.explained_variance_ratio_ # Percentage of variance explained by each of the selected components

In [ ]:
# visualize first two components
fig, ax = plt.subplots(figsize=(5, 4))
scatter = ax.scatter(ld1, ld2, c=labels, cmap=sorghum_cmap, s=20, alpha=0.7)
hs, _ = scatter.legend_elements() # handlers of the legend
ls = ['background','leaf','stalk','panicle'] # labels of the legend
ax.legend(hs, ls)
ax.set_xlabel('LD1 (%.2f)'%lda_scores[0])
ax.set_ylabel('LD2 (%.2f)'%lda_scores[1])
plt.tight_layout()
#plt.savefig('lda.png', dpi=150)
plt.show()

- compare the PCA and LDA differences
![images on cyverse](notebook_imgs/lda.png)

### SVM (support vector machine)

- learn more details about [SVM](https://scikit-learn.org/stable/modules/svm.html) on scikit-learn website

In [ ]:
from sklearn import svm

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=123)
for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    clf = svm.SVC(kernel='poly') # try linear kernel also
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    acc = (y_predict==y_test).sum()/len(y_test)
    print('overal accuracy: %.3f'%acc)
    cfx = confusion_matrix(y_test, y_predict, labels=[0,1,2,3], normalize='true')
    print('accuracy in each class: ',np.diagonal(cfx))

In [ ]:
plot_cfx(cfx)

### KNN (k-nearest neighbors)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN

- learn more details about [KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) on scikit-learn website

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=123)
for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    clf = KNN(n_neighbors=3) # default is 5
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    acc = (y_predict==y_test).sum()/len(y_test)
    print('overal accuracy: %.3f'%acc)
    cfx = confusion_matrix(y_test, y_predict, labels=[0,1,2,3], normalize='true')
    print('accuracy in each class: ',np.diagonal(cfx))
    
## plot confusion matrix

In [ ]:
plot_cfx(cfx)

### random forest

- learn more details about [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) on scikit-learn website

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RF

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=123)
for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    clf = RF() # try parameters: n_estimators=200, max_depth=5
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    acc = (y_predict==y_test).sum()/len(y_test)
    print('overal accuracy: %.3f'%acc)
    cfx = confusion_matrix(y_test, y_predict, labels=[0,1,2,3], normalize='true')
    print('accuracy in each class: ',np.diagonal(cfx))    

In [ ]:
plot_cfx(cfx)

- check which features are more imporant

In [ ]:
clf = RF()
clf.fit(features, labels)
gini = clf.feature_importances_ # Gini importance. The higher, the more important of the feature.

In [ ]:
# plot gini importance across wavelengths
fig, ax = plt.subplots(figsize=(4, 3))
ax.plot(df_wave['Wavelength(nm)'], gini)
ax.set_xlabel('Wavelength (nm)', fontsize=12)
ax.set_ylabel('Gini importance', fontsize=12)

plt.tight_layout()
#plt.savefig('RF_Gini.png', dpi=150)
plt.show()

![images on cyverse](notebook_imgs/RF_Gini.png)

### Try [Native Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) and [AdaBoost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) classifiers yourself

In [ ]:
### Native Bayes
from sklearn.naive_bayes import GaussianNB

# train and evaluate your model

# show the confusion matrix

In [ ]:
## AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

# train and evaluate your model

# show the confusion matrix

## Make predictions on a hyperspectral image cube

- seems LAD performs best among the tested classifiers

In [ ]:
# load the hyperspectral image cube in numpy array
img_npy = np.load('NPYs/CM024_2017-08-30.npy')[35:478, :, :]/255 # remove the pot and top frame parts
img_npy.shape

- remember the input dimension of our model is (N, 243) where N is the number of pixels

In [ ]:
# train LDA model using all the training data
clf = LDA()
clf.fit(features, labels)

# make predictions
x, y, z = img_npy.shape
x_test = img_npy.reshape(x*y, z) # reshape 3d (443, 320, 243) to 2d (141760, 243) 
y_test = clf.predict(x_test).reshape(x, y) # reshape (141760, ) back to (443, 320) to make picture
print(y_test.shape)

In [ ]:
# show our prediction results
cmap = ListedColormap(['#FFFFFF', '#32CD32', '#FF8C00', '#9400D3'], name='test')
plt.imshow(y_test, cmap=cmap)
# plt.imsave('test.png', y_test, cmap=cmap) # run if you want to save the prediction as a png file

original vs predicted:
![images on cyverse](notebook_imgs/test_prediction1.png)

### try to make predictions on other two hyperspectral image cubes under the 'NPYs' directory

![images on cyverse](notebook_imgs/test_prediction2.png)

## Estiamte organ sizes from the predictions

- other traits requiring image processing skills will not be covered here

In [ ]:
size_leaf = (y_test==1).sum()  # number of leaf pixels
size_stalk = (y_test==2).sum() # number of stalk pixels
size_panicle = (y_test==3).sum() # number of panicle pixels

print(size_leaf, size_stalk, size_panicle)

## Hyperspectral data in numpy format and Genotype data in hmp format for a sorghum diverse population

- Download all image data from [Cyverse](https://datacommons.cyverse.org/browse/iplant/home/shared/commons_repo/curated/Miao_Schnable_sorghumHighThroughputPhenotyping_2017)

![images on cyverse](notebook_imgs/cyverse.png)

- Download genotype data from [figshare](https://doi.org/10.6084/m9.figshare.11462469.v5)
![figshare genotype](notebook_imgs/figshare.png)

## To cite

Cite this paper if you use the hyperspectral image data for the sorghum association panel:
- Chenyong Miao, Alejandro Pages, Zheng Xu, Eric Rodene, Jinliang Yang, and James C. Schnable (2020) Semantic segmentation of sorghum using hyperspectral data identifies genetic associations. **Plant Phenomics** doi: 10.34133/2020/4216373

Please cite this paper if you use the genotype data for the same population
- Chenyong Miao, Yuhang Xu, Sanzhen Liu, Patrick S. Schnable, and James C. Schnable (2020) Increased power and accuracy of locus identification in time-series genome-wide association in sorghum. **Plant Physiology** doi: 10.1101/2020.02.16.951467